In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 47.3 MB 1.6 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=c8cbbf4c1f2823f2fd9506b70368ef5339b87d9b2817ea3d1908a7cdefe2ddab
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 8.9 MB/s 
     |████████████████████████████████| 754 kB 8.3 MB/s 
     |████████████████████████████████| 3.0 MB 53.9 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-1tycoi4k
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-1tycoi4k
     |████████████████████████████████| 132 kB 8.3 MB/s 
     |████████████████████████████████| 4.5 MB 64.3 MB/s 
     |████████████████████████████████| 3.8 MB 55.3 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 6.5 MB 46.6 MB/s 
     |████████████████████████████████| 596 kB 54.0 MB/s 
     |████████████████████████████████| 8.6 MB 33.8 MB/s 
     |████████████████████████████████| 79 kB 4.4 MB/s 
     |████████████████████████████████| 127 kB 59.1 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=06c54daaf061da3bb9ee7ead37ac51a7797f961de82df692bd8a456a746aca86
  Stored in directory: /tmp/pip-ephem-wheel-cache-3_5zah_9/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
path = '/content/drive/MyDrive/통계데이터센터/'

train_data = pd.read_csv( path + 'train_preprocessing_aug.csv',encoding='cp949')
test_data = pd.read_csv( path + 'test_preprocessing.csv',encoding='cp949')

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train_data['digit_3'] = encoder.fit_transform(train_data['digit_3'].values)

In [ ]:
display(encoder.classes_)

array([ 11,  12,  14,  20,  31,  32,  51,  61,  62,  71,  72,  80, 101,
       102, 103, 104, 105, 106, 107, 108, 111, 112, 120, 131, 132, 133,
       134, 139, 141, 142, 143, 144, 151, 152, 161, 162, 163, 171, 172,
       179, 181, 182, 191, 192, 201, 202, 203, 204, 205, 211, 212, 213,
       221, 222, 231, 232, 233, 239, 241, 242, 243, 251, 252, 259, 261,
       262, 263, 264, 265, 266, 271, 272, 273, 274, 281, 282, 283, 284,
       285, 289, 291, 292, 301, 302, 303, 304, 311, 312, 313, 319, 320,
       331, 332, 333, 334, 339, 340, 351, 352, 353, 360, 370, 381, 382,
       383, 390, 411, 412, 421, 422, 423, 424, 425, 426, 451, 452, 453,
       461, 462, 463, 464, 465, 466, 467, 468, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 491, 492, 493, 494, 495, 501, 502, 511, 512,
       521, 529, 551, 559, 561, 562, 581, 582, 591, 592, 601, 602, 611,
       612, 620, 631, 639, 641, 642, 649, 651, 652, 653, 661, 662, 681,
       682, 701, 702, 711, 712, 713, 714, 715, 716, 721, 729, 73

In [ ]:
train_data

,digit_3,text
0,221,카센터에서 자동차 부분정비 타이어 오일 교환
1,126,상점 내에서 일반인을 대상으로 채소 과일 판매
2,123,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매
3,129,영업점에서 일반 소비자에게 열쇠 잠금장치
4,212,어린이집 보호자의 위탁을 받아 취학 전 아동보육
...,...,...
702213,206,교육 서비스업 교육 지원 교육지원 기타 서비스업 서비스업
702214,207,및 보건업 사회복지 서비스업 병원 종합병원
702215,207,보건업 및 사회복지 서비스업 치과병원
702216,209,보건업 및 사회복지 보건 공중 서비스업 의료업 공중 보건 의료업


In [ ]:
train_dataset = []
for sen, label in zip(train_data['text'], train_data['digit_3']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  train_dataset.append(data_train)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=225,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.4).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    # test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/21945 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 5.355643272399902 train acc 0.0
epoch 1 batch id 201 loss 5.463472366333008 train acc 0.005441542288557214
epoch 1 batch id 401 loss 5.240377426147461 train acc 0.01137780548628429
epoch 1 batch id 601 loss 5.1410698890686035 train acc 0.034525790349417634
epoch 1 batch id 801 loss 4.850860118865967 train acc 0.049079275905118604
epoch 1 batch id 1001 loss 4.373568534851074 train acc 0.05919080919080919
epoch 1 batch id 1201 loss 4.103190898895264 train acc 0.0739748126561199
epoch 1 batch id 1401 loss 3.755030870437622 train acc 0.09796573875802998
epoch 1 batch id 1601 loss 3.2910420894622803 train acc 0.12261867582760774
epoch 1 batch id 1801 loss 3.21764874458313 train acc 0.1488582731815658
epoch 1 batch id 2001 loss 3.128342866897583 train acc 0.17525612193903048
epoch 1 batch id 2201 loss 3.1244056224823 train acc 0.20037766924125397
epoch 1 batch id 2401 loss 2.924560070037842 train acc 0.22386505622657227
epoch 1 batch id 2601 loss 2.222273826599121 tra

  0%|          | 0/21945 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 2.203277826309204 train acc 0.46875
epoch 2 batch id 201 loss 0.5738503336906433 train acc 0.8369092039800995
epoch 2 batch id 401 loss 0.40848684310913086 train acc 0.8535692019950125
epoch 2 batch id 601 loss 0.2694515287876129 train acc 0.8592450083194676
epoch 2 batch id 801 loss 0.6220994591712952 train acc 0.8646613607990012
epoch 2 batch id 1001 loss 0.4823819398880005 train acc 0.8657592407592407
epoch 2 batch id 1201 loss 0.4539095163345337 train acc 0.8670639050791007
epoch 2 batch id 1401 loss 0.5426046848297119 train acc 0.8681968236973591
epoch 2 batch id 1601 loss 0.845091700553894 train acc 0.8695737039350406
epoch 2 batch id 1801 loss 0.6732946634292603 train acc 0.8695863409217102
epoch 2 batch id 2001 loss 0.8176669478416443 train acc 0.8699400299850075
epoch 2 batch id 2201 loss 0.5091763734817505 train acc 0.8700590640617901
epoch 2 batch id 2401 loss 0.44316989183425903 train acc 0.8698198667221991
epoch 2 batch id 2601 loss 0.27098244428634

...

epoch 3 batch id 1 loss 1.0727397203445435 train acc 0.78125
epoch 3 batch id 201 loss 0.25931379199028015 train acc 0.8826181592039801
epoch 3 batch id 401 loss 0.2594548761844635 train acc 0.8913653366583542
epoch 3 batch id 601 loss 0.23218633234500885 train acc 0.8948627287853578
epoch 3 batch id 801 loss 0.35199615359306335 train acc 0.8978620474406991
epoch 3 batch id 1001 loss 0.31461867690086365 train acc 0.8972902097902098
epoch 3 batch id 1201 loss 0.3380604386329651 train acc 0.8976894254787677
epoch 3 batch id 1401 loss 0.5370854735374451 train acc 0.8981754104211278
epoch 3 batch id 1601 loss 0.8033714294433594 train acc 0.89914506558401
epoch 3 batch id 1801 loss 0.4937330484390259 train acc 0.8989450305385897
epoch 3 batch id 2001 loss 0.8866227269172668 train acc 0.8989567716141929
epoch 3 batch id 2201 loss 0.5649081468582153 train acc 0.899108359836438
epoch 3 batch id 2401 loss 0.42667096853256226 train acc 0.898609954185756
epoch 3 batch id 2601 loss 0.2110715657472

  0%|          | 0/21945 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.8897591829299927 train acc 0.84375
epoch 4 batch id 201 loss 0.18155905604362488 train acc 0.9059390547263682
epoch 4 batch id 401 loss 0.16199809312820435 train acc 0.9130299251870324
epoch 4 batch id 601 loss 0.1040167510509491 train acc 0.9159733777038269
epoch 4 batch id 801 loss 0.4038721024990082 train acc 0.9186563670411985
epoch 4 batch id 1001 loss 0.28983575105667114 train acc 0.917800949050949
epoch 4 batch id 1201 loss 0.21189160645008087 train acc 0.918479392173189
epoch 4 batch id 1401 loss 0.4507473409175873 train acc 0.9191425767309065
epoch 4 batch id 1601 loss 0.6771413683891296 train acc 0.919737663960025
epoch 4 batch id 1801 loss 0.2457796186208725 train acc 0.9194891726818434
epoch 4 batch id 2001 loss 0.727405309677124 train acc 0.9195714642678661
epoch 4 batch id 2201 loss 0.5590073466300964 train acc 0.9195394139027715
epoch 4 batch id 2401 loss 0.3300918936729431 train acc 0.9189530403998334
epoch 4 batch id 2601 loss 0.13937771320343

  0%|          | 0/21945 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.5871239900588989 train acc 0.90625
epoch 5 batch id 201 loss 0.1199398785829544 train acc 0.9134017412935324
epoch 5 batch id 401 loss 0.08217200636863708 train acc 0.9239401496259352
epoch 5 batch id 601 loss 0.08552885800600052 train acc 0.9280886023294509
epoch 5 batch id 801 loss 0.2841814458370209 train acc 0.9312968164794008
epoch 5 batch id 1001 loss 0.16894447803497314 train acc 0.9311625874125874
epoch 5 batch id 1201 loss 0.08567184209823608 train acc 0.9322960033305578
epoch 5 batch id 1401 loss 0.5043004155158997 train acc 0.9329496788008566
epoch 5 batch id 1601 loss 0.6649681329727173 train acc 0.9337328232354778
epoch 5 batch id 1801 loss 0.19897757470607758 train acc 0.9339082454192116
epoch 5 batch id 2001 loss 0.6657683849334717 train acc 0.9342984757621189
epoch 5 batch id 2201 loss 0.577766478061676 train acc 0.9343338255338483
epoch 5 batch id 2401 loss 0.2506254017353058 train acc 0.9339858392336526
epoch 5 batch id 2601 loss 0.1814351081

In [ ]:
torch.save(model, path + 'model/kobert_중복제거_aug_e5.pt')

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predictproba(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for inp in out:
        p = inp.detach().cpu().numpy()

  return p

In [ ]:
preds = []
step = 0
for i in test_data['text']:
  preds.append(predictproba(i))

In [ ]:
preds = np.array(preds)
np.save(path + 'proba/kobert.npy',arr=preds)